## warm up exercise 

The catplots on page 3.3.4 are bad!

Problems:
1. Hard to
2. No titles on either graph 
3. missing data on the x-axsis

Fixes:
1. 

In [1]:
import pandas as pd
left_df = pd.DataFrame({
                    'firm': ['Accenture','Citi','GS'],
                    'varA': ['A1', 'A2', 'A3']})

right_df = pd.DataFrame({
                    'firm': ['GS','Chase','WF'],
                    'varB': ['B1', 'B2', 'B3'],
                    'varc': ['C1', 'C2', 'C3']})

In [2]:
left_df

,firm,varA
0,Accenture,A1
1,Citi,A2
2,GS,A3


In [3]:
right_df

,firm,varB,varc
0,GS,B1,C1
1,Chase,B2,C2
2,WF,B3,C3


## Merging Rules

1. Always use `on`!

In [4]:
pd.merge(left_df, right_df, 
         on='firm',  # on = 'firm' if one variable, on = ['firm', 'year']
         how='right' # adding variables from left to the right dataset, if a key isnt in the dataset, output from merge fills in with MISSING
        )    

# "how" controls which observations are in the post-merge dataset


# left,
#     right,
#     how: str = 'inner',
#     on=None,
#     left_on=None,
#     right_on=None,
#     left_index: bool = False,
#     right_index: bool = False,
#     sort: bool = False,
#     suffixes=('_x', '_y'),
#     copy: bool = True,
#     indicator: bool = False,
#     validate=None,

,firm,varA,varB,varc
0,GS,A3,B1,C1
1,Chase,NaN,B2,C2
2,WF,NaN,B3,C3


# not all merges are the same!

# pay attention tot he uniqueness of the keys! second dataset not all keys are unique!

In [5]:
left_df2 = pd.DataFrame({
                    'firm': ['Accenture','Citi','GS'],
                    'varA': ['A1', 'A2', 'A3']})

right_df2 = pd.DataFrame({
                    'firm': ['GS','GS','WF'],
                    'varB': ['B1', 'B2', 'B3'],
                    'varc': ['C1', 'C2', 'C3']})

pd.merge(left_df2,right_df2,on='firm',how='left')

,firm,varA,varB,varc
0,Accenture,A1,NaN,NaN
1,Citi,A2,NaN,NaN
2,GS,A3,B1,C1
3,GS,A3,B2,C2


** "dont name dataset temp"

In [6]:
temp = pd.merge(left_df, right_df, on='firm',how='outer'    ,indicator=True , validate= 'one_to_one'   )  
temp['_merge'].value_counts()

right_only    2
left_only     2
both          1
Name: _merge, dtype: int64

### `Validate`

## Types of Merges (in "data wrangling skills" part of textbook)
1:1: The keys in each dataset are all unique 

1:M: The keys in right dataset have some duplicates m="many"

M:1: The keys in left dataset have some duplicates

M:1: The keys in both datasets have some duplicates

- NEVER USE THE MANY TO MANY OPTION 
    - always use the "smallest" option you think is correct: If youre doing a 1:1, type one_to_one
- Errors are good!
    - Either you're merging "wrong" (picked the wrong "on" vars),
    - The data has problems (duplicates where there should'nt be),
    - You misunderstand the data
    - M:M can be dangerous 

### `how`

typical use case for merging: working on some data and want to analyze, and want to merge in (add variable).


HOT TIP:  
1. **The number of rows in your dataset should'nt change!** Check len before and after!
2. How: 'left' 
    - new length = len(unmatched) + len(matched)
3. if we do a 1:1 --> any observations with matched to the right, wont have 2+ matching in the right, and will not "grow"
4. if we do M:1 --> any obs from the left data can only be matched to 1 obs from right --> therefore, number of obs wont grow
```` python
# a way to check 
len(df)
df = df.merge(someNewData,
              on=[listofvars],
              how='',
              validate='')
len(df)
````
              


### Creating Variables around a merge 

Poll: should we create vairable BEFORE (yes) the merge or AFTER (no)

```python
#before
newvar = function(right['x'])
left.merge(newVar)

# after
merge = left.merge(right)
merge['newVar'] = functioon(merge['x'])

```

## Missing values
These pandas functions can be very useful:

`fillna` - any value (strings included) you want, back fill, forward, fill, and more

`dropna` - might not be explicitly needed, some functions ignore NaNs, but some don't. I tend to drop only as needed for a given estimation, and only temporarily.

`replace` - some datasets use a certain number to indicate missing data (i.e. -99). Convert these to NaN with this function.

`interpolate` - e.g. use values in surrounding time periods to fill in gaps

`Deduce`. Suppose PPE this year is missing. $PPE_t = PPE_{t-1} + CAPX_t - DP_t $


In [7]:
import pandas as pd
import numpy as np

df = pd.DataFrame({"A":[12, 4, 5, None, 1], 
                   "B":[None, 2, 54, 3, None], 
                   "C":[20, 16, None, 3, 8], 
                   "D":[14, 3, None, None, 6]}) 

_df1 = df.copy()
_df1['firm'] = 1
_df1['date'] = _df1.index

_df2 = df.copy()
_df2['firm'] = 2
_df2['date'] = _df2.index

df2 = _df1.append(_df2)

## Questions for df:

    Fill all missing values with -1
    Fill missing values for variable "B" with -1
    Fill all values with the mean for the variable
    Fill all values with the median for the variable
    Fill values by taking the most recent non-missing prior value
    
    

## Questions for df2:

    Carry missing values forward without carrying values from firm 1 to firm 2
    Fill missing values with the average for firms on that date

In [8]:
# Q 1
df.fillna(-1)

,A,B,C,D
0,12.0,-1.0,20.0,14.0
1,4.0,2.0,16.0,3.0
2,5.0,54.0,-1.0,-1.0
3,-1.0,3.0,3.0,-1.0
4,1.0,-1.0,8.0,6.0


In [9]:
# Q 2 
df['B'].fillna(-1)

0    -1.0
1     2.0
2    54.0
3     3.0
4    -1.0
Name: B, dtype: float64

In [10]:
# Q 3
df.fillna(value = df.mean())

,A,B,C,D
0,12.0,19.666667,20.00,14.000000
1,4.0,2.000000,16.00,3.000000
2,5.0,54.000000,11.75,7.666667
3,5.5,3.000000,3.00,7.666667
4,1.0,19.666667,8.00,6.000000


In [11]:
# Q 4
df.fillna(value = df.median())

,A,B,C,D
0,12.0,3.0,20.0,14.0
1,4.0,2.0,16.0,3.0
2,5.0,54.0,12.0,6.0
3,4.5,3.0,3.0,6.0
4,1.0,3.0,8.0,6.0


In [12]:
#q5 Fill values by taking the most recent non-missing prior value

df.ffill()

,A,B,C,D
0,12.0,NaN,20.0,14.0
1,4.0,2.0,16.0,3.0
2,5.0,54.0,16.0,3.0
3,5.0,3.0,3.0,3.0
4,1.0,3.0,8.0,6.0


In [13]:
#q6 Carry missing values forward without carrying values from firm 1 to firm 2

df3 = df2.groupby(['firm']).ffill()
df3

,A,B,C,D,date
0,12.0,NaN,20.0,14.0,0
1,4.0,2.0,16.0,3.0,1
2,5.0,54.0,16.0,3.0,2
3,5.0,3.0,3.0,3.0,3
4,1.0,3.0,8.0,6.0,4
0,12.0,NaN,20.0,14.0,0
1,4.0,2.0,16.0,3.0,1
2,5.0,54.0,16.0,3.0,2
3,5.0,3.0,3.0,3.0,3
4,1.0,3.0,8.0,6.0,4


In [14]:
#q7 Fill missing values with the average for firms on that date

df2.groupby('date').mean() # dave this, merge in on date! (but dont overwrite existing NaNs)


# (
# df2.groupby('date')
# .fillna(df2.grouby('firm'))
# )    

,A,B,C,D,firm
date,,,,,
0,12.0,NaN,20.0,14.0,1.5
1,4.0,2.0,16.0,3.0,1.5
2,5.0,54.0,NaN,NaN,1.5
3,NaN,3.0,3.0,NaN,1.5
4,1.0,NaN,8.0,6.0,1.5


### Outliers (3.3.1 in textbook)

In [15]:
# copied from 3.3.4.1

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# these three are used to download the file
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

url = 'https://github.com/LeDataSciFi/ledatascifi-2021/blob/main/data/CCM_cleaned_for_class.zip?raw=true'

#firms = pd.read_stata(url)   
# <-- that code would work, but GH said it was too big and
# forced me to zip it, so here is the work around to download it:

with urlopen(url) as request:
    data = BytesIO(request.read())

with ZipFile(data) as archive:
    with archive.open(archive.namelist()[0]) as stata:
        ccm = pd.read_stata(stata)

In [16]:
ccm.columns
vars_to_check = ['1_a','prof_a','mb', 'ppe_a', 'capx_a']

for v in vars_to_check:
    sns.kdeplot(x= (ccm[v]-ccm[v].mean())/ccm[v].std())

KeyError: '1_a'

In [17]:
vars_to_check = ['l_a', 'l_sale', 'prof_a', 'mb', 'ppe_a', 'capx_a', 'xrd_a', 'cash_a', 'div_d', 'td_a', 'td_mv', 'dltt_a', 'dv_a', 	'invopps_FG09', 'sales_g', 'short_debt', 'long_debt_dum', 'atr', 'smalltaxlosscarry', 'largetaxlosscarry', 'tnic3hhi', 'tnic3tsimm', 'prodmktfluid', 'delaycon', 'equitydelaycon', 'debtdelaycon', 'privdelaycon', 'l_emp', 'l_ppent', 'l_laborratio']

std = (ccm[vars_to_check]-ccm[vars_to_check].mean() ) / ccm[vars_to_check].std()

std.describe(percentiles=[.01, .05, .1, .25, .5, .75, .90, .99])

,l_a,l_sale,prof_a,mb,ppe_a,capx_a,xrd_a,cash_a,div_d,td_a,...,tnic3hhi,tnic3tsimm,prodmktfluid,delaycon,equitydelaycon,debtdelaycon,privdelaycon,l_emp,l_ppent,l_laborratio
count,222978.000000,218779.000000,2.171670e+05,219269.000000,218189.000000,206399.000000,223001.000000,2.223320e+05,206688.000000,221468.000000,...,96951.000000,96951.000000,88332.000000,5.713000e+04,57130.000000,5.713000e+04,57130.000000,208991.000000,2.182120e+05,199821.000000
mean,-0.000001,0.000021,-2.393331e-07,0.000003,0.000011,0.000007,-0.000111,-5.218496e-07,0.000107,0.000007,...,0.000010,0.000004,0.000017,4.354530e-07,0.000002,2.279176e-07,-0.000001,-0.000005,-5.688926e-08,0.000002
std,1.000002,1.000002,1.000022e+00,1.000350,0.999983,0.999992,0.999853,9.999995e-01,1.001241,1.000288,...,0.999986,1.000006,0.999996,9.999976e-01,0.999996,1.000001e+00,1.000001,1.000000,9.999590e-01,0.999993
min,-5.038009,-4.775811,-1.879088e+02,-0.197607,-1.090727,-23.300566,-2.599572,-1.123044e+00,-0.907077,-0.783106,...,-0.966308,-0.510284,-1.913105,-2.856582e+00,-2.940712,-3.087471e+00,-3.145257,-0.891766,-1.455006e+00,-6.399152
1%,-1.997626,-2.633651,-9.574285e-01,-0.145698,-1.090727,-0.708069,-0.251713,-7.842962e-01,-0.907077,-0.649797,...,-0.915834,-0.456139,-1.532486,-1.921187e+00,-1.838654,-1.959609e+00,-1.930963,-0.891766,-1.455006e+00,-2.043165
5%,-1.551229,-1.631154,-3.379692e-01,-0.125848,-1.064053,-0.706551,-0.251713,-7.671428e-01,-0.907077,-0.649797,...,-0.852095,-0.454538,-1.279758,-1.482765e+00,-1.428327,-1.474460e+00,-1.476244,-0.880651,-1.373409e+00,-1.357046
10%,-1.265624,-1.207287,-1.542499e-01,-0.115641,-1.032479,-0.679735,-0.251713,-7.450486e-01,-0.907077,-0.649797,...,-0.801063,-0.451726,-1.104094,-1.213189e+00,-1.175825,-1.181956e+00,-1.199357,-0.858015,-1.219294e+00,-1.079315
25%,-0.726541,-0.627081,-2.553243e-02,-0.101243,-0.844128,-0.538163,-0.251713,-6.660880e-01,-0.907077,-0.521486,...,-0.670299,-0.438301,-0.729692,-7.184392e-01,-0.713450,-6.931342e-01,-0.702254,-0.762565,-8.009999e-01,-0.634864
50%,-0.055015,-0.006244,3.957242e-02,-0.076536,-0.289022,-0.268454,-0.251713,-4.213901e-01,-0.907077,-0.126007,...,-0.377095,-0.376669,-0.176650,-9.281078e-02,-0.113647,-1.051857e-01,-0.095857,-0.391239,-1.444730e-01,-0.131639
75%,0.679680,0.655297,9.638985e-02,-0.007981,0.580115,0.183921,-0.090278,2.533138e-01,1.103647,0.326616,...,0.280165,-0.149755,0.551248,6.254590e-01,0.575714,6.021436e-01,0.590432,0.449048,6.552986e-01,0.475080


### Options for dealing with them
1. fix (look at the data, correct, can costly)
2. Censor (delete)
3. Winsorize (change/cheap ad-doc correction that downplays the "WEIGHT" od the outlier in your analysis)

### Finding outliers

how can we find outliers?
1. scatterplot / hexplot
1. box plot
1. z-scores 
1. Plot density functions  


### Dealing with them one way

1. if <p1 --> p1
2. if >p99 --> p99 
    
Options: 
    - do this once 

In [19]:
# winsorizse one var at a time 
for v in vars_to_check:
	practice_df[v] = winsorize(practice_df[v],limits=[.01,.99])

	# one line equivalent:

	practice_df[vars_to_check]= practice_df[vars_to_check].apply(lambda x: winsorize(x,limits=[.01,.99]))

	# what if outliers in some years start at "different" points than other years
	# winsorize BY YEAR

	practice_df[vars_to_check] = practice_df.groupby('fyear')[vars_to_check].transform(lambda x: winsorize(x,limits=[.01,.99]))

NameError: name 'winsorize' is not defined

In [20]:
import pandas as pd
ccm_merge = pd.merge(ccm, patentstock, how='left', on=['gvkey', 'fyear'], validate='one_to_one', indicator=True)

NameError: name 'patentstock' is not defined

In [21]:
import seaborn as sns
import matplotlib.pyplot as plt
ax = sns.scatterplot(data = ccm_merge, x='capx_a', y='l_a')
plt.show()

NameError: name 'ccm_merge' is not defined